In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import janitor
import json
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

from utilities.utils import get_fulldata
from indicate import transliterate

2023-05-06 18:21:58.249386: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 18:21:58.292889: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 18:21:58.293985: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 18:21:59.032395: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## SHRUG

In [2]:
gdf_state = gpd.read_file("geometries_shrug-v1.5.samosa-open-polygons-gpkg/state.gpkg")
gdf_state.info()
bihar_state_id = gdf_state.set_index("state_name").at["Bihar", "pc11_state_id"]
gdf_state

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   pc11_state_id  35 non-null     object  
 1   state_name     35 non-null     object  
 2   geometry       35 non-null     geometry
dtypes: geometry(1), object(2)
memory usage: 968.0+ bytes


,pc11_state_id,state_name,geometry
0,01,Jammu and Kashmir,"POLYGON ((77.95837 35.48178, 77.96405 35.48433..."
1,02,Himachal Pradesh,"POLYGON ((76.80943 33.23872, 76.81593 33.23535..."
2,03,Punjab,"POLYGON ((75.83876 32.52156, 75.83898 32.52128..."
3,04,Chandigarh,"POLYGON ((76.79191 30.77115, 76.79229 30.77118..."
4,05,Uttarakhand,"POLYGON ((79.22439 31.34099, 79.22624 31.33888..."
5,06,Haryana,"POLYGON ((76.84307 30.88633, 76.84365 30.88618..."
6,07,NCT Of Delhi,"POLYGON ((77.07688 28.88184, 77.07801 28.88149..."
7,08,Rajasthan,"POLYGON ((73.90898 30.05334, 73.90437 30.05000..."
8,09,Uttar Pradesh,"MULTIPOLYGON (((79.36095 25.13890, 79.36217 25..."
9,10,Bihar,"MULTIPOLYGON (((84.51370 24.25774, 84.51182 24..."


In [3]:
gdf_districts = (
    gpd.read_file("geometries_shrug-v1.5.samosa-open-polygons-gpkg/district.gpkg")
    .query(f"pc11_state_id=='{bihar_state_id}'")
    .assign(district_name=lambda df: df["district_name"].str.lower())
    .reset_index(drop=True)
)
gdf_districts.info()
gdf_districts

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   pc11_state_id     38 non-null     object  
 1   pc11_district_id  38 non-null     object  
 2   district_name     38 non-null     object  
 3   geometry          38 non-null     geometry
dtypes: geometry(1), object(3)
memory usage: 1.3+ KB


,pc11_state_id,pc11_district_id,district_name,geometry
0,10,203,pashchim champaran,"POLYGON ((84.12469 27.51105, 84.12628 27.51085..."
1,10,204,purba champaran,"POLYGON ((84.83041 27.01919, 84.83128 27.01916..."
2,10,205,sheohar,"POLYGON ((85.32738 26.65375, 85.32803 26.65304..."
3,10,206,sitamarhi,"POLYGON ((85.66200 26.84586, 85.66625 26.84437..."
4,10,207,madhubani,"POLYGON ((86.06059 26.65650, 86.06645 26.65505..."
5,10,208,supaul,"POLYGON ((87.05293 26.55341, 87.05334 26.55334..."
6,10,209,araria,"POLYGON ((87.06975 26.56275, 87.06950 26.55288..."
7,10,210,kishanganj,"MULTIPOLYGON (((87.84553 25.98449, 87.84566 25..."
8,10,211,purnia,"POLYGON ((87.71802 26.11298, 87.71835 26.11297..."
9,10,212,katihar,"POLYGON ((87.82502 25.88259, 87.82553 25.88247..."


In [4]:
# Divisions?
gdf_subdistricts = (
    gpd.read_file("geometries_shrug-v1.5.samosa-open-polygons-gpkg/subdistrict.gpkg")
    .query(f"pc11_state_id=='{bihar_state_id}'")
    .assign(subdistrict_name=lambda df: df["subdistrict_name"].str.lower())
    .reset_index(drop=True)
)
gdf_subdistricts.info()
gdf_subdistricts.head(5)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   pc11_state_id        534 non-null    object  
 1   pc11_district_id     534 non-null    object  
 2   pc11_subdistrict_id  534 non-null    object  
 3   subdistrict_name     534 non-null    object  
 4   geometry             534 non-null    geometry
dtypes: geometry(1), object(4)
memory usage: 21.0+ KB


,pc11_state_id,pc11_district_id,pc11_subdistrict_id,subdistrict_name,geometry
0,10,203,01013,sidhaw,"POLYGON ((83.94735 27.44297, 83.94897 27.44205..."
1,10,203,01014,ramnagar,"POLYGON ((84.12469 27.51105, 84.12628 27.51085..."
2,10,203,01015,gaunaha,"POLYGON ((84.38576 27.37299, 84.39084 27.36994..."
3,10,203,01016,mainatanr,"POLYGON ((84.65097 27.29101, 84.65572 27.28294..."
4,10,203,01017,narkatiaganj,"POLYGON ((84.58513 27.19071, 84.58565 27.19002..."


In [5]:
gdf_villages = (
    gpd.read_file("geometries_shrug-v1.5.samosa-open-polygons-gpkg/village.gpkg")
    .query(f"pc11_state_id=='{bihar_state_id}'")
    .assign(town_village_name=lambda df: df["town_village_name"].str.lower())
    .reset_index(drop=True)
)
gdf_villages.info()
gdf_villages.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 45107 entries, 0 to 45106
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   pc11_state_id         45107 non-null  object  
 1   pc11_district_id      45107 non-null  object  
 2   pc11_subdistrict_id   45107 non-null  object  
 3   pc11_town_village_id  45107 non-null  object  
 4   town_village_name     45072 non-null  object  
 5   geometry              45107 non-null  geometry
dtypes: geometry(1), object(5)
memory usage: 2.1+ MB


,pc11_state_id,pc11_district_id,pc11_subdistrict_id,pc11_town_village_id,town_village_name,geometry
0,10,203,01013,215989,kalapani,"POLYGON ((83.95195 27.44241, 83.95057 27.44029..."
1,10,203,01013,215990,bhaisalotan,"POLYGON ((83.94735 27.44297, 83.94446 27.44172..."
2,10,203,01013,215991,tharhi,"POLYGON ((83.86745 27.43616, 83.87093 27.43540..."
3,10,203,01013,215992,pipra,"POLYGON ((83.90969 27.40805, 83.90759 27.40606..."
4,10,203,01013,215993,kotaraha,"POLYGON ((83.91060 27.39674, 83.90982 27.39623..."


## Divisions (subdistricts?): Land records

* https://github.com/soodoku/land/blob/main/scripts/00_summary_basic_bihar_land_record.ipynb

In [6]:
fp_divisions = "divisions.csv"
if False:
    # Get divisions
    df_divisions = (
        get_fulldata(**{'usecols': ["division"]})
        .drop_duplicates(ignore_index=True)
        .assign(eng=np.nan)
    )
    # Transliterate
    for ix, row in tqdm(df_divisions.iterrows()):
        df_divisions.at[ix, "eng"] = transliterate.hindi2english(row["division"])
        
    df_divisions.to_csv(fp_divisions, index=False)
    
df_divisions = pd.read_csv(fp_divisions)
df_divisions.head(3)

,division,eng
0,वैशाली,vaishali
1,पटेढ़ी बेलसर,patedhi beluser
2,लालगंज,lalganj


In [7]:
df_divisions[df_divisions.duplicated("eng", keep=False)]

,division,eng
83,संग्रमपुर,sangrampur
106,बाराहाट,barahat
177,बरहट,barahat
268,सा0 कमाल,NaN
276,चे0 बरियारपुर,NaN
366,बगहा -1,NaN
450,संग्रामपुर,sangrampur


In [8]:
# Try merge with SHRUG
df_divisions = (
    df_divisions
    .dropna(subset=["eng"])
    .drop_duplicates("eng")
    .merge(gdf_subdistricts, how="outer", left_on="eng", right_on="subdistrict_name", validate="1:m", indicator=True)
)
df_divisions

,division,eng,pc11_state_id,pc11_district_id,pc11_subdistrict_id,subdistrict_name,geometry,_merge
0,वैशाली,vaishali,10,220,01268,vaishali,"POLYGON ((85.17616 26.00890, 85.17655 26.00858...",both
1,पटेढ़ी बेलसर,patedhi beluser,NaN,NaN,NaN,NaN,None,left_only
2,लालगंज,lalganj,10,220,01270,lalganj,"POLYGON ((85.27043 25.91443, 85.26983 25.91342...",both
3,भगवानपुर,bhagwanpur,10,220,01271,bhagwanpur,"POLYGON ((85.28554 25.93287, 85.28608 25.93210...",both
4,भगवानपुर,bhagwanpur,10,222,01308,bhagwanpur,"POLYGON ((85.97766 25.62317, 85.97795 25.62281...",both
...,...,...,...,...,...,...,...,...
811,NaN,NaN,10,239,01537,kako,"POLYGON ((85.09374 25.28850, 85.09469 25.28781...",right_only
812,NaN,NaN,10,239,01538,modanganj,"POLYGON ((85.14576 25.31181, 85.14654 25.31180...",right_only
813,NaN,NaN,10,239,01540,makhdumpur,"POLYGON ((85.07797 25.15016, 85.07815 25.14854...",right_only
814,NaN,NaN,10,240,01542,arwal,"POLYGON ((84.70671 25.31483, 84.70909 25.31016...",right_only


In [9]:
df_divisions.query("_merge=='both'")

,division,eng,pc11_state_id,pc11_district_id,pc11_subdistrict_id,subdistrict_name,geometry,_merge
0,वैशाली,vaishali,10,220,01268,vaishali,"POLYGON ((85.17616 26.00890, 85.17655 26.00858...",both
2,लालगंज,lalganj,10,220,01270,lalganj,"POLYGON ((85.27043 25.91443, 85.26983 25.91342...",both
3,भगवानपुर,bhagwanpur,10,220,01271,bhagwanpur,"POLYGON ((85.28554 25.93287, 85.28608 25.93210...",both
4,भगवानपुर,bhagwanpur,10,222,01308,bhagwanpur,"POLYGON ((85.97766 25.62317, 85.97795 25.62281...",both
5,भगवानपुर,bhagwanpur,10,233,01455,bhagwanpur,"POLYGON ((83.62819 25.01638, 83.62923 25.01636...",both
...,...,...,...,...,...,...,...,...
503,औराई,aurai,10,216,01207,aurai,"POLYGON ((85.58092 26.39322, 85.58173 26.39088...",both
504,कटरा,katra,10,216,01208,katra,"POLYGON ((85.65221 26.32562, 85.65257 26.32386...",both
505,काँटी,kanti,10,216,01204,kanti,"MULTIPOLYGON (((85.38662 26.14974, 85.38624 26...",both
508,साहेबगंज,sahebganj,10,216,01199,sahebganj,"POLYGON ((85.01314 26.32546, 85.01443 26.32402...",both


In [10]:
df_divisions.query("_merge=='left_only'")

,division,eng,pc11_state_id,pc11_district_id,pc11_subdistrict_id,subdistrict_name,geometry,_merge
1,पटेढ़ी बेलसर,patedhi beluser,NaN,NaN,NaN,NaN,None,left_only
7,विदुपुर,vidupur,NaN,NaN,NaN,NaN,None,left_only
8,राधोपुर,radhopur,NaN,NaN,NaN,NaN,None,left_only
12,चेहरा कलॉ,chehra kalau,NaN,NaN,NaN,NaN,None,left_only
13,राजापाकर,rajapakar,NaN,NaN,NaN,NaN,None,left_only
...,...,...,...,...,...,...,...,...
500,बंदरा,bandara,NaN,NaN,NaN,NaN,None,left_only
501,बोचहाँ,bochahan,NaN,NaN,NaN,NaN,None,left_only
506,पारु,paru,NaN,NaN,NaN,NaN,None,left_only
507,मोतीपुर,motipur,NaN,NaN,NaN,NaN,None,left_only


In [11]:
df_divisions.query("_merge=='right_only'")

,division,eng,pc11_state_id,pc11_district_id,pc11_subdistrict_id,subdistrict_name,geometry,_merge
511,NaN,NaN,10,203,01013,sidhaw,"POLYGON ((83.94735 27.44297, 83.94897 27.44205...",right_only
512,NaN,NaN,10,203,01014,ramnagar,"POLYGON ((84.12469 27.51105, 84.12628 27.51085...",right_only
513,NaN,NaN,10,203,01016,mainatanr,"POLYGON ((84.65097 27.29101, 84.65572 27.28294...",right_only
514,NaN,NaN,10,203,01017,narkatiaganj,"POLYGON ((84.58513 27.19071, 84.58565 27.19002...",right_only
515,NaN,NaN,10,203,01018,lauriya,"POLYGON ((84.37649 27.06930, 84.37713 27.06922...",right_only
...,...,...,...,...,...,...,...,...
811,NaN,NaN,10,239,01537,kako,"POLYGON ((85.09374 25.28850, 85.09469 25.28781...",right_only
812,NaN,NaN,10,239,01538,modanganj,"POLYGON ((85.14576 25.31181, 85.14654 25.31180...",right_only
813,NaN,NaN,10,239,01540,makhdumpur,"POLYGON ((85.07797 25.15016, 85.07815 25.14854...",right_only
814,NaN,NaN,10,240,01542,arwal,"POLYGON ((84.70671 25.31483, 84.70909 25.31016...",right_only


## Districts: Land records

In [12]:
fp_districts = "districts.csv"
if False:
    # Get district
    df_districts = (
        get_fulldata(**{'usecols': ["district"]})
        .drop_duplicates(ignore_index=True)
        .assign(eng=np.nan)
    )
    # Transliterate
    for ix, row in tqdm(df_districts.iterrows()):
        df_districts.at[ix, "eng"] = transliterate.hindi2english(row["district"])
        
    df_districts.to_csv(fp_districts, index=False)
    
df_districts = pd.read_csv(fp_districts)
df_districts.head(3)

,district,eng
0,वैशाली,vaishali
1,गया,gaya
2,रोहतास,rohtas


In [13]:
assert len(df_districts) == len(gdf_districts)

In [14]:
# Try merge with SHRUG
df_districts = (
    df_districts
#     .dropna(subset=["eng"])
#     .drop_duplicates("eng")
    .merge(gdf_districts, how="outer", left_on="eng", right_on="district_name", validate="1:1", indicator=True)
    .sort_values("_merge")
    .reset_index(drop=True)
    .rename_column("district", "bihar_lr_district")
    .rename_column("eng", "bihar_lr_district_eng")
    .rename_column("district_name", "shrug_district")
)
df_districts

,bihar_lr_district,bihar_lr_district_eng,pc11_state_id,pc11_district_id,shrug_district,geometry,_merge
0,पूर्णियॉं,purniyaaayaaon,NaN,NaN,NaN,None,left_only
1,खगड़िया,khagriya,NaN,NaN,NaN,None,left_only
2,सीतामढ़ी,sitamadhi,NaN,NaN,NaN,None,left_only
3,कैमूर,camor,NaN,NaN,NaN,None,left_only
4,पूर्वी चम्पारण,purvi champaran,NaN,NaN,NaN,None,left_only
5,किशनगंज,kishenganj,NaN,NaN,NaN,None,left_only
6,अरवल,araval,NaN,NaN,NaN,None,left_only
7,बेगुसराए,begusaray,NaN,NaN,NaN,None,left_only
8,मुज़फ्फरपुर,muzfarpur,NaN,NaN,NaN,None,left_only
9,नवादा,navada,NaN,NaN,NaN,None,left_only


In [15]:
# Show successful merges
(df_districts
 .sort_values("_merge")
 .query("_merge=='both'")
 .remove_columns(["pc11_state_id", "geometry"])
 .reset_index(drop=True)
)

,bihar_lr_district,bihar_lr_district_eng,pc11_district_id,shrug_district,_merge
0,दरभंगा,darbhanga,215,darbhanga,both
1,पटना,patna,230,patna,both
2,मधेपुरा,madhepura,213,madhepura,both
3,कटिहार,katihar,212,katihar,both
4,सारण,saran,219,saran,both
5,गोपालगंज,gopalganj,217,gopalganj,both
6,लखीसराय,lakhisarai,227,lakhisarai,both
7,भोजपुर,bhojpur,231,bhojpur,both
8,औरंगाबाद,aurangabad,235,aurangabad,both
9,सहरसा,saharsa,214,saharsa,both


In [16]:
# Show only districts found in Bihar land record
(df_districts
 .sort_values("_merge")
 .query("_merge=='left_only'")
 .remove_columns(["pc11_state_id", "geometry"])
 .reset_index(drop=True)
)

,bihar_lr_district,bihar_lr_district_eng,pc11_district_id,shrug_district,_merge
0,पूर्णियॉं,purniyaaayaaon,NaN,NaN,left_only
1,बक्सर,bucksre,NaN,NaN,left_only
2,शिवहर,shivahar,NaN,NaN,left_only
3,सिवान,sivan,NaN,NaN,left_only
4,शेखपुरा,shekhpura,NaN,NaN,left_only
5,नवादा,navada,NaN,NaN,left_only
6,मुज़फ्फरपुर,muzfarpur,NaN,NaN,left_only
7,जहानाबाद,jahanabad,NaN,NaN,left_only
8,अरवल,araval,NaN,NaN,left_only
9,किशनगंज,kishenganj,NaN,NaN,left_only


In [17]:
# Show only districts found in SHRUG
(df_districts
 .sort_values("_merge")
 .query("_merge=='right_only'")
 .remove_columns(["pc11_state_id", "geometry"])
 .reset_index(drop=True)
)

,bihar_lr_district,bihar_lr_district_eng,pc11_district_id,shrug_district,_merge
0,NaN,NaN,232,buxar,right_only
1,NaN,NaN,239,jehanabad,right_only
2,NaN,NaN,240,arwal,right_only
3,NaN,NaN,210,kishanganj,right_only
4,NaN,NaN,237,nawada,right_only
5,NaN,NaN,233,kaimur (bhabua),right_only
6,NaN,NaN,228,sheikhpura,right_only
7,NaN,NaN,211,purnia,right_only
8,NaN,NaN,222,begusarai,right_only
9,NaN,NaN,218,siwan,right_only


## Residence (villages?): Land Records

In [18]:
fp_residences = "residences.csv"
if True:
    # Get district
    df_residences = (
        get_fulldata(**{'usecols': ["district"]})
        .drop_duplicates(ignore_index=True)
        .assign(eng=np.nan)
    )
    # Transliterate
    for ix, row in tqdm(df_residences.iterrows()):
        df_residences.at[ix, "eng"] = transliterate.hindi2english(row["district"])
        
    df_residences.to_csv(fp_residences, index=False)
    
df_residences = pd.read_csv(fp_residences)
df_residences.head(3)

0it [00:00, ?it/s]

2023-05-06 18:26:35.571296: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-06 18:26:35.571611: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


,district,eng
0,वैशाली,vaishali
1,गया,gaya
2,रोहतास,rohtas
